# Introdução

Na primeira etapa dos experimentos de finetuning, nós fizemos um total de 20 experimentos, variando:
 - tamanho do modelo (`t5-base` e `t5-small`)
 - como tratar caracteres `\n`: substituindo por espaço (` `) ou pipes (`|`)
 - modelo treinado sobre dataset com apenas uma tarefa ou todas (concatenação de datasets de todas tarefas)


# Initial setup

In [1]:
%load_ext autoreload
%autoreload 2

# Loading expriment data from Neptune

In [2]:
# Loading project data
import neptune
project = neptune.init('marcospiau/final-project-ia376j-1')
experiments = project.get_experiments()
display(experiments)

[Experiment(FIN-27),
 Experiment(FIN-28),
 Experiment(FIN-29),
 Experiment(FIN-30),
 Experiment(FIN-33),
 Experiment(FIN-34),
 Experiment(FIN-35),
 Experiment(FIN-36),
 Experiment(FIN-37),
 Experiment(FIN-38),
 Experiment(FIN-39),
 Experiment(FIN-40),
 Experiment(FIN-41),
 Experiment(FIN-42),
 Experiment(FIN-43),
 Experiment(FIN-44),
 Experiment(FIN-45),
 Experiment(FIN-46),
 Experiment(FIN-47),
 Experiment(FIN-48)]

In [4]:
leaderboard = project.get_leaderboard()
leaderboard.head()

,id,name,created,finished,owner,notes,running_time,size,tags,channel_best_model_path,...,property_max_epochs,property_min_epochs,property_param__generate_max_length,property_param__learning_rate,property_param__optimizer,property_param__t5_model_prefix,property_param__t5_tokenizer_prefix,property_patience,property_str_replace_newlines,property_task_train
0,FIN-27,t5-base_all_tasks_concat_newlines_as_pipes,2021-01-02 06:49:33.609000+00:00,2021-01-02 15:58:31.521000+00:00,marcospiau,,32937,13603174,"[t5-base, all_tasks_concat, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-base,t5-base,50,|,all_tasks_concat
1,FIN-28,t5-base_all_tasks_concat_newlines_as_spaces,2021-01-02 15:58:50.605000+00:00,2021-01-02 22:02:48.959000+00:00,marcospiau,,21838,9068745,"[t5-base, all_tasks_concat, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-base,t5-base,50,,all_tasks_concat
2,FIN-29,t5-small_all_tasks_concat_newlines_as_pipes,2021-01-02 22:03:04.051000+00:00,2021-01-03 01:38:50.889000+00:00,marcospiau,,12946,5292147,"[t5-small, all_tasks_concat, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-small,t5-small,50,|,all_tasks_concat
3,FIN-30,t5-small_all_tasks_concat_newlines_as_spaces,2021-01-03 01:39:06.081000+00:00,2021-01-03 03:38:56.588000+00:00,marcospiau,,7190,2940305,"[t5-small, all_tasks_concat, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-small,t5-small,50,,all_tasks_concat
4,FIN-33,t5-base_extract_company_newlines_as_pipes,2021-01-03 04:02:04.774000+00:00,2021-01-03 05:35:03.390000+00:00,marcospiau,,5578,2446937,"[t5-base, extract_company, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-base,t5-base,50,|,extract_company


In [5]:
[x.name for x in experiments]

['t5-base_all_tasks_concat_newlines_as_pipes',
 't5-base_all_tasks_concat_newlines_as_spaces',
 't5-small_all_tasks_concat_newlines_as_pipes',
 't5-small_all_tasks_concat_newlines_as_spaces',
 't5-base_extract_company_newlines_as_pipes',
 't5-base_extract_company_newlines_as_spaces',
 't5-small_extract_company_newlines_as_pipes',
 't5-small_extract_company_newlines_as_spaces',
 't5-base_extract_total_newlines_as_pipes',
 't5-base_extract_total_newlines_as_spaces',
 't5-small_extract_total_newlines_as_pipes',
 't5-small_extract_total_newlines_as_spaces',
 't5-base_extract_address_newlines_as_pipes',
 't5-base_extract_address_newlines_as_spaces',
 't5-small_extract_address_newlines_as_pipes',
 't5-small_extract_address_newlines_as_spaces',
 't5-base_extract_date_newlines_as_pipes',
 't5-base_extract_date_newlines_as_spaces',
 't5-small_extract_date_newlines_as_pipes',
 't5-small_extract_date_newlines_as_spaces']

In [8]:
from src.data.sroie import get_all_keynames_from_dir
from src.data.sroie import get_dataloaders_dict_from_datasets_dict
from src.data.sroie import get_tasks_functions_maps
from src.data.sroie.t5_ocr_baseline import get_datasets_dict_from_task_functions_map
from src.data.sroie.t5_ocr_baseline import get_default_preprocessing_functions
from src.models.t5_ocr_baseline import T5OCRBaseline
from functools import partial
from tqdm import tqdm
import gc
import pytorch_lightning as pl
import gin
import tempfile
import multiprocessing as mp
import os
import torch
import pandas as pd

gin.enter_interactive_mode()
gin.clear_config()

In [48]:
project.get_experiments('FIN-28')[0]

Experiment(FIN-28)

In [9]:
gin.parse_config_file(
    '/home/marcospiau/final_project_ia376j/src/models/gin/defaults.gin',
    skip_unknown=True
)
with gin.config_scope('train_sroie'):
    train_keynames = get_all_keynames_from_dir()

with gin.config_scope('validation_sroie'):
    val_keynames = get_all_keynames_from_dir()

gin.clear_config()

In [10]:
# experiments_to_eval = project.get_experiments(tag='all_tasks_concat')
experiments_to_eval = project.get_experiments()
display([(experiment.id, experiment.name) for experiment in experiments_to_eval])

[('FIN-27', 't5-base_all_tasks_concat_newlines_as_pipes'),
 ('FIN-28', 't5-base_all_tasks_concat_newlines_as_spaces'),
 ('FIN-29', 't5-small_all_tasks_concat_newlines_as_pipes'),
 ('FIN-30', 't5-small_all_tasks_concat_newlines_as_spaces'),
 ('FIN-33', 't5-base_extract_company_newlines_as_pipes'),
 ('FIN-34', 't5-base_extract_company_newlines_as_spaces'),
 ('FIN-35', 't5-small_extract_company_newlines_as_pipes'),
 ('FIN-36', 't5-small_extract_company_newlines_as_spaces'),
 ('FIN-37', 't5-base_extract_total_newlines_as_pipes'),
 ('FIN-38', 't5-base_extract_total_newlines_as_spaces'),
 ('FIN-39', 't5-small_extract_total_newlines_as_pipes'),
 ('FIN-40', 't5-small_extract_total_newlines_as_spaces'),
 ('FIN-41', 't5-base_extract_address_newlines_as_pipes'),
 ('FIN-42', 't5-base_extract_address_newlines_as_spaces'),
 ('FIN-43', 't5-small_extract_address_newlines_as_pipes'),
 ('FIN-44', 't5-small_extract_address_newlines_as_spaces'),
 ('FIN-45', 't5-base_extract_date_newlines_as_pipes'),
 ('FI

In [35]:
# Temporary dicts for metric saving
train_metrics = dict()
val_metrics = dict()
val_metrics

{}

In [ ]:
get_all_keynames_from_dir

In [36]:
for experiment in experiments_to_eval:
    print(120 * '*')
    print(f'Evaluating experiment {experiment.id}')
    
    gin.clear_config()
#     Loading experiment data
    experiment_channels = experiment.get_channels()
    experiment_properties = experiment.get_properties()
    
#     Loading best checkpoint
    model = T5OCRBaseline.load_from_checkpoint(
        experiment_channels['best_model_path'].y
    )
    trainer = pl.Trainer(gpus=1, logger=False)
    
#     Datasets and dataloaders
    with tempfile.TemporaryDirectory() as tmp_folder:
        experiment.download_artifact('gin_operative_config.gin', tmp_folder)
        gin.parse_config_file(
            os.path.join(tmp_folder, 'gin_operative_config.gin'),
            skip_unknown=True
        )

    with gin.config_scope('sroie_t5_baseline'):
        task_functions_maps = get_tasks_functions_maps()

    train_datasets = get_datasets_dict_from_task_functions_map(
        keynames=train_keynames, tasks_functions_maps=task_functions_maps)
    val_datasets = get_datasets_dict_from_task_functions_map(
        keynames=val_keynames, tasks_functions_maps=task_functions_maps)

    loader_kwargs = {
        'num_workers': mp.cpu_count(),
        'shuffle': False,
        'pin_memory': True
    }

    train_dataloaders = get_dataloaders_dict_from_datasets_dict(
        datasets_dict=train_datasets, dataloader_kwargs=loader_kwargs)
    val_dataloaders = get_dataloaders_dict_from_datasets_dict(
        datasets_dict=val_datasets, dataloader_kwargs=loader_kwargs)
    
#     Metrics on train and validation data
    assert train_dataloaders.keys() == val_dataloaders.keys()
    train_metrics[experiment.id] = dict()
    val_metrics[experiment.id] = dict()
    for task_eval in val_dataloaders.keys():
        train_metrics[experiment.id][task_eval] = trainer.test(
            model,
            test_dataloaders=train_dataloaders[task_eval]
        )
        val_metrics[experiment.id][task_eval] = trainer.test(
            model,
            test_dataloaders=val_dataloaders[task_eval]
        )
    
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    break

************************************************************************************************************************
Evaluating experiment FIN-27


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2021-01-11 02:49:59 | INFO | lightning | GPU available: True, used: True
2021-01-11 02:49:59 | INFO | lightning | TPU available: None, using: 0 TPU cores
2021-01-11 02:49:59 | INFO | lightning | LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


----------------------------------------------------------------------------------------------------
Sample predictions epoch  0 'test':
keyname: /home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/train/X51005663280
formatted_input:
    What is the address? 31812012|Chili's|Chili's Grill & Bar KLCC|T.A.S Leisure Sdn Bhd (256864-P)|Tel:03-2164 1400|GST ID:001042104320|Tax Receipt|GST ID:001042104320|4 CHILIS3|23 Anis|Gst4|Chk 1080323|30 Dec 17 18:40:57|TBL64 / 1|30/12/17|-- EAT IN ---|1 PINEAPPLE JUICE|1 TOSTADA CHIPS|1 CHP_BWL CK|1 MARG CK|12.95 S|16.95 S|29.95 S|27.95 5|Subtotal|10% Srv Chg|GST @6%|Total:|87.90|8.78|5.81|102.39|0.01|Total 102.40|YOUR OPINION MATTERS 102.40|We invite you to complete our|GUEST EXPERIENCE SURVEY|YOU COULD WIN RM500 CASH OR|CHILI'S VOUCHER FOR_RM1,000|A WINNER EVERY MONTH|Visit www.go-chilis.com|YOUR SURVEY CODE:5013|Pis enter within the next 4 days|Must be 18 or above. Void where|prohibited. See web for complete|rules and sweepstakes detail

----------------------------------------------------------------------------------------------------
Sample predictions epoch  0 'test':
keyname: /home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/test/X51007339157
formatted_input:
    What is the address? SANYU STATIONERY SHOP|NO. 316&33G, JALAN SETIA INDAH X ,U13/X|40170 SETIA ALAM|Mobile /Whatsapp : +6012-918 7937|Tel: +603-3362 4137|GST ID No: 001531760640|TAX INVOICE|Owned By :|SANYU SUPPLY SDN BHD (1135772-K)|CASH SALES COUNTER|1. 5000-0001|PHOTOCOPY SERVICES - A4|SIZE|50 X 0.1000|5.00|SR|Total Sales Inclusive GST @6%|5.00|Discount|0.00|Total|Round Adj|5.00|0.00|Final Total|5.00|CASH|CHANGE|5.00|0.00|GST Summary|SR @ 6%|Amount(RM)|4.72|Tax(RM)|0.28|INV NO: CS-SA-0097493|Date : 19/07/2017|Goods sold are not Returnable & Refundable|THANK YOU FOR YOUR PATRONAGE|PLEASE COME AGAIN.|TERIMA KASIH SILA DATANG LAGI|** PLEASE KEEP THIS RECEIPT FOR PROVE OF|PURCHASE DATE FOR IT PRODUCT WARRANTY|PURPOSE **|Follow us in Facebook

----------------------------------------------------------------------------------------------------
Sample predictions epoch  0 'test':
keyname: /home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/train/X51005568829
formatted_input:
    What is the date? THAI DELICIOUS RESTAURANT|NO.22 JALAN PUTERI 2/4|BANDAR PUTERI|47100 PUCHONG, SELANGOR|03-8051 9514|Bill No 01-36870|Table|Date : 02/02/2017 7:31:08 PM АЗ|Cashier : 123|Pax|:|Qty|Price|20.00|Amount|20.00|Description|06S BRAISED|PORKLEG SMALL|ขายเล็ก|2 32 PHRANAKON|SALTED EGG|SQUID minim|3 51 GARLIC FRIED|KAILAN in|28.00|28.00|13.00|13.00|4|1|84 THAI STYLE|KERABU TOFU|15.00|15.00|4|1|1.80|6.50|7.20|6.50|5 130 PLAIN RICE|6 113 COCONUT|THAI|7 116 DISTILLED|WATER|8 106B COLD|PANDAN GINGER|TEA|1|0.60|0 60|1|2.50|2.50|Total|92.80|Grand Total:|Cash Received|Change|92.80|100.00|7.20|AVERAGE/PAX(S):|23.20|Thank You. Please Come Again.|
formatted_labels:
    02/02/2017
generated_text:
    02/02/2017
precision: 1.0
recall: 1.0
f1: 1

...

----------------------------------------------------------------------------------------------------
Sample predictions epoch  0 'test':
keyname: /home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/test/X51008099054
formatted_input:
    What is the total? 10 GRAM Gourmet Sbn Bhd|(1152264-K)|No 3, Jalan Temenggung 27/9,|Bandar Mahkota Cheras,|43200 Selangor.|(GST Reg. No : 002055098368)|Tax Invoice|Table -|Check #: 514658|Pax(s): 0|Date : 11-06-2018 12:51:34|Cashier: Cashier Morning|Description Qty|U.price Total TAX|PO2 Spaghetti Aglio Olio Chicken Breast|1 X 15.00 15.00 SR|Total (Excluding GST):|Total GST (0%):|Total (Inclusive of GST):|15.00|0.00|15.00|TOTAL:|15.00|Closed: 8888 11-06-2018 12:52:21|Server: Cashier Morning|CASH :|15.00|GST Summary|SR (@0%)|Amount (RM). Tax(RM)|15.00 0.00|**** Thank You ****|Please Come Again|
formatted_labels:
    15.00
generated_text:
    15.00
precision: 1.0
recall: 1.0
f1: 1.0
exact_match: 1


--------------------------------------------

----------------------------------------------------------------------------------------------------
Sample predictions epoch  0 'test':
keyname: /home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/train/X51005444045
formatted_input:
    What is the company? 99 SPEED MART S/B (519537-X)|LOT P.T. 2811, JALAN ANGSA,|TAMAN BERKELEY|41150 KLANG, SELANGOR|1245-DESA SRI HARTAMAS|GST ID. NO : 000181747712|INVOICE NO : 18341/103/T0138|09:06PM|569547|19-03-18|103 SUNQUICK OREN 840ML|RM11.40 s|Total Sales (Inclusive GST) RM 11.40|CASH RM 11.40|CHANGE RM .00|--------|GST Susuary|Aaount(RM)|10.75|Tax(RM)|.65|$ - 67|Thank You. Please come again|Keep the invoice for applicable returns|
formatted_labels:
    99 SPEED MART S/B
generated_text:
    99 SPEED MART S/B
precision: 1.0
recall: 1.0
f1: 1.0
exact_match: 1


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_exact_match': 0.9929577464788732,
 'test_f1': 0.99862135777628

----------------------------------------------------------------------------------------------------
Sample predictions epoch  0 'test':
keyname: /home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/test/X51007843145
formatted_input:
    What is the company? t|A06140|TAX INVOICE|BHPetrol Permas Jaya 2|LOT PTD 101051|Jalan Permas 10/10|81750 Masai, Johor|Esjay Fuel Enterprise|CO. REG. NO: 001011128-T|GST REG. NO: 000153358336|30-06-2018 09:09:49 Cashier: 465|Invoice No: 1-642195|Prepay Chit Number: 12091|Item|U\Price|Qnty Amount|Inf RON95|P3 2.200|22.73 50.00 R|TOTAL SALES|ROUNDING ADJ|RM 50.00|RM 0.00|TOTAL AMOUNT PAYABLE|RM 50.00|CASH|RM 50.00|Prepay Deposit RM 50.00|Prepay Refund RM 0.00|GST SUMMARY|Amount RM Tax RM|R: Relief*|50.00 0.00|*Relief under Section 56(3)(b)GST Act 2014|Thank You For Shopping At BHPetromart|Customer Service 1-300-22-2828|:|
formatted_labels:
    ESJAY FUEL ENTERPRISE
generated_text:
    BHPETROL PERMAS JAYA 2
precision: 0
recall: 0
f1: 0
exact_m

----------------------------------------------------------------------------------------------------
Sample predictions epoch  0 'test':
keyname: /home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/train/X51006557202
formatted_input:
    What is the address? GARDENIA BAKERIES (KL) SDN BHD (139386 X)|Lot 3, Jalan Pelabur 23/1,|40300 Shah Alam, Selangor.|Tel: 03 55423228 Fax:03. 55423213|GSTID: 00038 1399040|TAX INVOICE / ADJUSTMENT NOTE|Cash In No.: 7006F710|VEOS|Date: 08/10/2017|MAKASSAR FRESH MARKET SDN BHD|GROUND FLOOR, NO. 4 & 6,|JALAN SS 15/4B,|47500 SUBANG JAYA, SELANGOR|VEOS: Ridzuan (11900)|DD: 08/10/2017 12:11|Description U.P Iss Exc D Sale Amt(RM)|0.C. WHITE 2.13 9 2 © 7|WHOLEMEAL 2.78 6|4 11.12|0.C JUMBO 2.97 8 1|7|20.79|Total 0% supplies: 46.82|CR-8' SCOTCH 0.72|-1 -0.72|Total 6% supplies (excl. GST): -0.72|GST: -0.04|Total 6% supplies (Inc. GST): -0.76|Total 0% supplies: 46.82|Total Payable:|46.06|E.&.(|Peceived above goods in good order condition.|e recipient 

----------------------------------------------------------------------------------------------------
Sample predictions epoch  0 'test':
keyname: /home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/test/X51006913023
formatted_input:
    What is the company? .|A05030|UNIHAKKA INTERNATIONAL SDN BHD|16 May 2018 18:30|(867388-U)|12, Jalan Tampoi 7/4, Kawasan Perindustrian|Tampoi, 81200 Johor Bahru, Johor|TAX INVOICE|Invoice #: OR18051602160350|Total|Item|Qty|SR 100100000060-4 Vege|$5.50|1|$5.50|SR 100100000001- Chicken|$2.70|1|$2.70|Total Amount: $8.20|GST @6%: $0.46|Nett Total: $8.20|Payment Mode|CASH|Change|Amount|$8.20|$0.00|Amount($)|Tax($)|GST Summary|SR = GST @6%|7.74|0.46|GST REG #000656195584|BAR WANG RICE@PERMAS JAYA|(Price Inclusive of GST)|Thank You & Come Again!|Like and Follow Us on Facebook!|Facebook.com/BarWangRice|-|i|
formatted_labels:
    UNIHAKKA INTERNATIONAL SDN BHD
generated_text:
    UNIHAKKA INTERNATIONAL SDN BHD
precision: 1.0
recall: 1.0
f1: 1.0
exact

In [ ]:
train_metrics

In [37]:
train_metrics['FIN-27'] == train_metrics['FIN-28']

KeyError: 'FIN-28'

In [21]:
train_metrics.keys()

dict_keys(['FIN-27', 'FIN-28', 'FIN-29', 'FIN-30', 'FIN-33', 'FIN-34', 'FIN-35', 'FIN-36', 'FIN-37', 'FIN-38', 'FIN-39', 'FIN-40', 'FIN-41', 'FIN-42', 'FIN-43', 'FIN-44', 'FIN-45', 'FIN-46', 'FIN-47', 'FIN-48'])

In [54]:
train_metrics['FIN-27']

{'extract_address': [{'test_precision': 0.9962133914598703,
   'test_recall': 0.9942773449815704,
   'test_f1': 0.9951551490946686,
   'test_exact_match': 0.9788732394366197}],
 'extract_date': [{'test_precision': 1.0,
   'test_recall': 1.0,
   'test_f1': 1.0,
   'test_exact_match': 1.0}],
 'extract_total': [{'test_precision': 0.9976525821596244,
   'test_recall': 0.9976525821596244,
   'test_f1': 0.9976525821596244,
   'test_exact_match': 0.9976525821596244}],
 'extract_company': [{'test_precision': 0.9992175273865416,
   'test_recall': 0.9981611893583725,
   'test_f1': 0.9986213577762874,
   'test_exact_match': 0.9929577464788732}],
 'all_tasks_concat': [{'test_precision': 0.9982708752515091,
   'test_recall': 0.9975227791248918,
   'test_f1': 0.9978572722576452,
   'test_exact_match': 0.9923708920187794}]}

In [53]:
val_metrics['FIN-27']

{'extract_address': [{'test_precision': 0.9023434602678024,
   'test_recall': 0.8892146107676294,
   'test_f1': 0.8941288971350287,
   'test_exact_match': 0.44}],
 'extract_date': [{'test_precision': 0.975,
   'test_recall': 0.975,
   'test_f1': 0.975,
   'test_exact_match': 0.945}],
 'extract_total': [{'test_precision': 0.895,
   'test_recall': 0.9075,
   'test_f1': 0.8983333333333334,
   'test_exact_match': 0.875}],
 'extract_company': [{'test_precision': 0.9302976190476191,
   'test_recall': 0.9141190476190476,
   'test_f1': 0.9184844322344321,
   'test_exact_match': 0.82}],
 'all_tasks_concat': [{'test_precision': 0.9256602698288554,
   'test_recall': 0.9214584145966692,
   'test_f1': 0.9214866656756985,
   'test_exact_match': 0.77}]}

In [ ]:
pd.DataFrame({k:v['extract_address'] for k,v in metrics_dict.items()})

In [ ]:
pd.DataFrame.from_records({k:v['extract_address'] for k,v in metrics_dict.items()})

In [ ]:
def metrics_dict_to_pandas(metrics_dict, prefix):
    df = pd.DataFrame.from_dict(
        metrics_dict.copy(), orient='index', columns=['tmp_results']
    )
    metrics = df.iloc[0,0].keys()
    for metric in metrics:
        df[metric] = df['tmp_results'].apply(lambda x: x[metric])

    df = df.drop('tmp_results', axis=1)
    df.columns = [x.replace('test_', prefix) for x in df.columns]
    df = df.reset_index().rename({'index': 'task_eval'}, axis=1)
    
    return df
metrics_dict_to_pandas(train_metrics['FIN-28'], 'train')

In [ ]:
metrics_dict_to_pandas(val_metrics['FIN-28'], 'val')

In [ ]:
a.iloc[0,0]

In [ ]:
pd.DataFrame.from_dict(metrics_dict['FIN-27'], orient='index')[0][0]

In [ ]:
pd.DataFrame(metrics_dict)

In [ ]:
{k:v['extract_address'] for k,v in metrics_dict.items()}

In [ ]:
dict((x.id, x.get_channels()['best_model_path'].y) for x in experiments_to_eval)

In [ ]:
metrics_dict['FIN-28']

In [ ]:
{k:v['extract_address'] for k,v in metrics_dict.items()}

In [ ]:
print(metrics_dict)